In [1]:
using FFTW, DelimitedFiles, Printf

# 処理するフォルダのパス
folder_path = raw"E:\20250226_解析データ\6.0Hz_8.0Lmin_C001H001S0002"
# 出力フォルダの基準パス
base_output_folder = raw"C:\Users\sigot\OneDrive\デスクトップ\hp0_lp5_int0.01667_6.0Hz_8.0Lmin_C001H001S0002_S0001"
# テキストデータ間隔（秒）
interval_sec = 1/60
# バンドパスフィルタの閾値（Hz）
highpass_threshold = 0  # 低周波カット
lowpass_threshold = 5   # 高周波カット

5

In [ ]:
# フィルタ前_速度要素FFTグラフ出力
# 指定フォルダ内のテキストファイルを取得
files = filter(x -> endswith(x, ".txt"), readdir(folder_path, join=true))
sort!(files)

timesteps = length(files)  # タイムステップ数
fs = 1 / interval_sec  # サンプリング周波数
freqs = fftfreq(timesteps, fs)

# データの格納用辞書
velocity_data = Dict()

# 各ファイルを読み込み、座標 (8,8) の速度データを取得
for file in files
    data = readdlm(file)
    for row in eachrow(data)
        x, y, vx, vy, speed = row[1:5]
        if (x, y) == (8.0, 8.0)
            if !haskey(velocity_data, (x, y))
                velocity_data[(x, y)] = Float64[]
            end
            push!(velocity_data[(x, y)], speed)
        end
    end
end

# FFTの計算
if haskey(velocity_data, (8.0, 8.0))
    vels = velocity_data[(8.0, 8.0)]
    fft_data = fft(vels)
    mag_spectrum = abs.(fft_data)

    # 両対数グラフのプロット
    plot(freqs[2:div(timesteps, 2)], mag_spectrum[2:div(timesteps, 2)],
         xlabel="Frequency (Hz)", ylabel="Magnitude", title="FFT of Original Velocities at (8,8)",
         label="Magnitude Spectrum", linewidth=2, xscale=:log10, yscale=:log10)
else
    println("No velocity data found for coordinate (8,8)")
end


In [ ]:
# フィルタ掛けおよびテキストデータ出力
# 指定フォルダ内のテキストファイルを取得
all_files = filter(x -> endswith(x, ".txt"), readdir(folder_path, join=true))

# ファイルが存在しない場合のエラーチェック
if isempty(all_files)
    error("No text files found in the specified folder: $folder_path")
end

# 共通部分（ファイル名の末尾6桁を除いた部分）を取得
function get_common_prefix_and_suffix(filename)
    base = splitext(basename(filename))[1]  # 拡張子除去
    if length(base) > 6
        return base[1:end-6], base[end-5:end]  # (共通部分, 可変部分)
    else
        return "", base  # 例外処理
    end
end

# すべてのファイルの共通部分を取得し、グループ化
file_groups = Dict{String, Vector{Tuple{String, String}}}()
for file in all_files
    prefix, suffix = get_common_prefix_and_suffix(file)
    if prefix != ""
        if !haskey(file_groups, prefix)
            file_groups[prefix] = []
        end
        push!(file_groups[prefix], (file, suffix))
    end
end

# ファイルグループが空の場合のエラーチェック
if isempty(file_groups)
    error("No valid file groups found in the specified folder.")
end

# 1つの共通部分に対応するファイルのみ処理（最初のグループを採用）
first_group = first(values(file_groups))  # 先頭のグループを取得
sort!(first_group, by = x -> x[2])  # 可変部分（番号順）でソート
common_prefix = first(first_group, 1)[1][1] |> get_common_prefix_and_suffix |> first  # 先頭ファイルの共通部分を取得

# フォルダ名を入力フォルダと同じ命名規則で作成（filteredなし）
folder_name = "hp$(highpass_threshold)_lp$(lowpass_threshold)_int$(round(interval_sec, digits=5))_$(common_prefix)"
output_folder = joinpath(base_output_folder, folder_name)

# フォルダが存在しない場合は作成
if !isdir(output_folder)
    mkdir(output_folder)
end

# データの格納用辞書
velocity_data = Dict()
coordinates = Set{Tuple{Float64, Float64}}()

timesteps = length(first_group)  # タイムステップ数

# 各ファイルを読み込み、速度データを格納
for (file, _) in first_group
    data = readdlm(file)
    for row in eachrow(data)
        x, y, vx, vy, speed = row[1:5]
        push!(coordinates, (x, y))
        if !haskey(velocity_data, (x, y))
            velocity_data[(x, y)] = Float64[]
        end
        push!(velocity_data[(x, y)], speed)
    end
end

# FFT、バンドパスフィルタ、逆FFTを適用
t_filtered = Dict()
fs = 1 / interval_sec  # サンプリング周波数
freqs = fftfreq(timesteps, fs)

for coord in coordinates
    vels = velocity_data[coord]
    fft_data = fft(vels)

    # バンドパスフィルタ適用
    fft_data[abs.(freqs) .< highpass_threshold] .= 0  # 低周波成分除去
    fft_data[abs.(freqs) .> lowpass_threshold] .= 0   # 高周波成分除去

    # 逆FFTで時間領域に戻す
    t_filtered[coord] = real(ifft(fft_data))
end

# フィルタ後のデータを新しいテキストファイルに出力（フォーマット維持）
for (i, (file, suffix)) in enumerate(first_group)
    data = readdlm(file)
    new_data = copy(data)
    for row in eachrow(new_data)
        x, y = row[1:2]
        row[5] = t_filtered[(x, y)][i]  # 速度値を更新
    end
    
    # 出力ファイル名を変更（filteredなし）
    output_filename = "hp$(highpass_threshold)_lp$(lowpass_threshold)_int$(round(interval_sec, digits=5))_$(common_prefix)_$(suffix).txt"
    output_file = joinpath(output_folder, output_filename)

    # テキストのフォーマットを統一
    open(output_file, "w") do f
        for row in eachrow(new_data)
            @printf(f,"%5d%5d%15.7e%15.7e%15.7e%15.7e%15.7e%15.7e\n",
                    row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8])
        end
    end
end

# 座標 (8,8) の処理結果をログ出力
if haskey(t_filtered, (8.0, 8.0))
    println("Original velocities at (8,8):", velocity_data[(8.0, 8.0)])
    println("Filtered velocities at (8,8):", t_filtered[(8.0, 8.0)])
end
